# Trabajo Practico 3 - Finanzas Computacionales (UdeSA)
## Bautista Remondino - Santiago Pini - Pedro Gallo - Ignacio Bidarte


# Ejercicio 1

Empezamos con el metodo explicito, este primer bloque muestra se configuro la grilla de $x$ e $y$, $\Delta \tau$ y $\lambda$ estable


In [48]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Parámetros del problema
K = 100.0
r = 0.05
sigma = 0.20
T = 1.0

grilla_x = np.linspace(10, 200, 100)
grilla_t = np.linspace(0, T, 100)

S_min, S_max = 0.1*K, 4.0*K  
x_min, x_max = np.log(S_min), np.log(S_max)

Nx  = 400                              # num de intervalos en x
dx  = (x_max - x_min)/Nx
x   = np.linspace(x_min, x_max, Nx+1)

tau_max = 0.5 * sigma**2 * T

# Para el EXPÍCITO necesito λ = Δτ / (Δx)^2 ≤ 0.5  → elijo λ=0.45
lam   = 0.45
dtau  = lam * dx**2
Nt    = int(np.ceil(tau_max / dtau))   # cantidad de pasos de tiempo
dtau  = tau_max / Nt                   
lam   = dtau / dx**2                   # λ final (≤ 0.5 garantizado)
tau   = np.linspace(0.0, tau_max, Nt+1)


### Condiciones iniciales y de frontera

In [49]:
# Condición inicial: valor de la opción al vencimiento (t=T, τ=0)
y = np.exp(-r*T) * np.maximum(np.exp(x) - K, 0)

# Condición de frontera izquierda: S → 0  (x → -∞ o x_min)
# Notamos que es una frontera constante, no depende del tiempo por lo cual la podemos dejar fija.
def frontera_izquierda(tau):
    return 0.0

# Condición de frontera derecha: S → ∞  (x → +∞ o x_max)
def frontera_derecha(tau):
    return ( np.exp( -r*(T - 2.0*tau/sigma**2) )
             * np.exp( x_max - (2.0*r/sigma**2 - 1.0)*tau )
             - K * np.exp(-r*T) )


### Metodo explícito

Partimos de la ecuación del calor:

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2}
$$

Reemplazando por derivadas centrada y adelantada:

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{y(x+1,t) - 2y(x,t) + y(x-1,t)}{\Delta x^2}
$$

Buscamos aislar $ y(x,t+1) $, por tanto multiplicamos a ambos lados $\Delta t$:

$$
y(x,t+1) - y(x,t) = \frac{\Delta t}{\Delta x^2} \left[ y(x+1,t) - 2y(x,t) + y(x-1,t) \right]
$$

Definimos $ \lambda = \frac{\Delta t}{\Delta x^2} $, entonces:

$$
y(x,t+1) = y(x,t) + \lambda \left[ y(x+1,t) - 2y(x,t) + y(x-1,t) \right]
$$

In [50]:
for n in range(Nt):
    y_new = y.copy()
    y_new[1:-1] = y[1:-1] + lam * (y[2:] - 2*y[1:-1] + y[:-2])
    # fronteras
    y_new[0]  = frontera_izquierda(tau[n+1])
    y_new[-1] = frontera_derecha(tau[n+1])
    y = y_new


# Tomamos un precio en especial y chequeamos el resultado numérico

shift = (2.0*r/sigma**2 - 1.0) * tau_max
S_grid = np.exp( x - shift )          # mapea x → S en t=0
V_num = y                             # en t=0 el factor exponencial es 1, luego V = y

S0 = K
idx = np.argmin( np.abs(S_grid - S0) )
V_S0 = V_num[idx]
print(f"Precio numérico explícito para S0={S0}, t=0: {V_S0}")

Precio numérico explícito para S0=100.0, t=0: 10.49231204305964


### Metodo Implícito

Nuevamente partimos de la ecuación del calor:

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2}
$$

Reemplazando por derivadas centrada y adelantada:

$$
\frac{y(x,t+1) - y(x,t)}{\Delta t} = \frac{y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1)}{\Delta x^2}
$$

Buscamos aislar $ y(x,t+1) $, por tanto multiplicamos a ambos lados $\Delta t$:

$$
y(x,t+1) - y(x,t) = \frac{\Delta t}{\Delta x^2} \left[ y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1) \right]
$$

Definimos $ \lambda = \frac{\Delta t}{\Delta x^2} $, entonces:

$$
y(x,t+1) = y(x,t) + \lambda \left[ y(x+1,t+1) - 2y(x,t+1) + y(x-1,t+1) \right]
$$

### 🧱 3️⃣ Agrupá las incógnitas (versión con \(x+1, x+2\) y \(t+\Delta t\))

Si definís el vector de incógnitas en los puntos interiores del dominio espacial:

$$
y(x, t + \Delta t) =
\begin{bmatrix}
y(x + \Delta x, t + \Delta t) \\
y(x + 2\Delta x, t + \Delta t) \\
y(x + 3\Delta x, t + \Delta t) \\
y(x + 4\Delta x, t + \Delta t)
\end{bmatrix},
\quad
y(x, t) =
\begin{bmatrix}
y(x + \Delta x, t) \\
y(x + 2\Delta x, t) \\
y(x + 3\Delta x, t) \\
y(x + 4\Delta x, t)
\end{bmatrix}.
$$

Entonces, el sistema de ecuaciones del método implícito se puede escribir como:

$$
A\,y(x, t + \Delta t) = y(x, t) + d(x, t),
$$

donde la matriz \(A\) tiene estructura tridiagonal:

$$
A =
\begin{bmatrix}
1 + 2\lambda & -\lambda & 0 & 0 \\
-\lambda & 1 + 2\lambda & -\lambda & 0 \\
0 & -\lambda & 1 + 2\lambda & -\lambda \\
0 & 0 & -\lambda & 1 + 2\lambda
\end{bmatrix},
$$

y el vector \(d(x,t)\) incorpora las condiciones de frontera:

$$
d(x, t) =
\begin{bmatrix}
\lambda\,y(x, t + \Delta t) \\[4pt]
0 \\[4pt]
0 \\[4pt]
\lambda\,y(x + 5\Delta x, t + \Delta t)
\end{bmatrix}.
$$

In [ ]:
main_diag = (1 + 2*lam) * np.ones(Nx-1)
off_diag  = -lam * np.ones(Nx-2)
A = np.diag(main_diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1)

### Chequeo con un función de una librería

In [26]:
from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta

S = 100
K = 100
r = 0.05
sigma = 0.20
t = 1.0
flag = 'c'   # 'c' para call, 'p' para put

price = black_scholes(flag, S, K, t, r, sigma)
print("Precio B-S:", price)
print("Delta:", delta(flag, S, K, t, r, sigma))
print("Gamma:", gamma(flag, S, K, t, r, sigma))

Precio B-S: 10.450583572185561
Delta: 0.6368306511756191
Gamma: 0.018762017345846895
